In [1]:
# Import Dependencies

from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from bs4 import BeautifulSoup as bs
import cssutils
import time
import pandas as pd
import requests

In [2]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "/usr/local/bin/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)
    # Windows USERS -> Uncommented these 2 lines below, and comment out the top 2 lines
    #executable_path = {'executable_path': 'resources/v74/chromedriver.exe'}
    #return Browser("chrome", **executable_path, headless=False)

In [3]:
# 1. Nasa Mars News

url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
response = requests.get(url)
soup = bs(response.text, 'html.parser')
article = soup.find('div', class_="slide")
news_title = article.find('div', class_='content_title').text
news_p = article.find('div', class_='rollover_description_inner').text
    
print(news_title)
print(news_p)



NASA Garners 7 Webby Award Nominations



Nominees include four JPL projects: the solar system and climate websites, InSight social media, and a 360-degree Earth video. Public voting closes April 18, 2019.



In [4]:
# 2. JPL Mars Space Images - Features Image
mars_space_url = 'https://www.jpl.nasa.gov'
mars_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

mars_image_response = requests.get(mars_image_url)
mars_soup = bs(mars_image_response.text, 'html.parser')
mars_div = mars_soup.find('article')['style']
mars_url = mars_div.split("('", 1)[1].split("')")[0]
# Scrape page into Soup
featured_image_url = f'{mars_space_url}{mars_url}'
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA20063-1920x1200.jpg


In [5]:
# 3. Mars Weather - Twitter
weather_url = "https://twitter.com/marswxreport?lang=en"
weather_response = requests.get(weather_url)
weather_soup = bs(weather_response.text, 'html.parser')
weather_tweet = weather_soup.find('p', class_="TweetTextSize").text

#news_title = article.find('div', class_='content_title').text

print(weather_tweet)
#mars_weather = 'Sol 1801 (Aug 30, 2017), Sunny, high -21C/-5F, low -80C/-112F, pressure at 8.82 hPa, daylight 06:09-17:55'

InSight sol 152 (2019-05-01) low -98.1ºC (-144.5ºF) high -17.2ºC (1.0ºF)
winds from the SW at 4.8 m/s (10.7 mph) gusting to 13.2 m/s (29.5 mph)
pressure at 7.40 hPapic.twitter.com/zJGo11uNF1


In [6]:
# 4 Mars Facts - table containing facts about the planet including Diameter, Mass, etc.

mars_facts_url = "https://space-facts.com/mars/"

mars_data = pd.read_html(mars_facts_url)
mars_data[0]

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [15]:
# 5 Mars Hemispheres - Use a Python dictionary to store the data using the keys img_url and title
# Create an Empty List to hold the dicitionary 
hemisphere_list = []
# Call the Function
browser = init_browser()
hemisphere_html = browser.html
hemisphere_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemisphere_url)
time.sleep(1)


html = browser.html
hem_soup_master = bs(html, 'html.parser')
items = hem_soup_master.find_all('div', class_='item')

for item in items:
    item_url = item.find('h3').text
    try:
        browser.find_link_by_partial_text(item_url)[0].click()
        time.sleep(3)
    except ElementDoesNotExist:
        print("Complete")
        
    html = browser.html
    hem_soup_two = bs(html, 'html.parser')
    hemisephere_image = hem_soup_two.find('img', class_='wide-image')['src']
    hemisphere_title = hem_soup_two.find('h2', class_='title').text
    
    hemisphere_dict = {}
    hemisphere_dict["img_url"] = "https://astrogeology.usgs.gov" + hemisephere_image
    hemisphere_dict["title"] = hemisphere_title
    hemisphere_list.append(hemisphere_dict)
    
    browser.back()

In [8]:
hemisphere_list

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [16]:
browser.quit()